In [ ]:
import pandas as pd
from neo4j import GraphDatabase
from time import sleep
from random import randint
import ast

In [2]:
# 1. Load CSV file and convert 'menu' and 'embedding' to list
# -----------------------------------------------------------------------------
df = pd.read_csv('wave_emb_small.csv')
df = df.drop(['Unnamed: 0'], axis=1)
print(f"Tipe data nilai pada kolom 'menu' sebelum konversi: {type(df['menu'][0])}\nTipe data nilai pada kolom 'embedding' sebelum konversi: {type(df['embedding'][0])}")
df['menu'] = df['menu'].apply(ast.literal_eval)
df['embedding'] = df['embedding'].apply(ast.literal_eval)
print(f"Tipe data nilai pada kolom 'menu' setelah konversi: {type(df['menu'][0])}\nTipe data nilai pada kolom 'embedding' setelah konversi: {type(df['embedding'][0])}")
df

Tipe data nilai pada kolom 'menu' sebelum konversi: <class 'str'>
Tipe data nilai pada kolom 'embedding' sebelum konversi: <class 'str'>
Tipe data nilai pada kolom 'menu' setelah konversi: <class 'list'>
Tipe data nilai pada kolom 'embedding' setelah konversi: <class 'list'>


,restaurant,menu,city,alt_resto,embedding
0,BP Kerupuk Sayur,"[Kerupuk Labu BP Kerupuk Sayur, Kerupuk Wortel...",Surabaya,['Dapur Fizzul'],"[[-0.019313940289107857, 0.026720912450162143,..."
1,Bumbu Rawon,"[Bumbu Rawon UKG (Usaha Kerta Gemilang), Bumbu...",Kabupaten Gresik,['Rawon Balungan H. Mufid'],"[[-0.0012182795813163079, 0.038871749931996775..."
2,JAMU BU SOLIKAH,"[JAMU KUNIR SURUH, JAMU KUNIR, JAMU BERAS KENC...",Malang,['Sambat Luwe'],"[[0.01761782175153306, -0.006151651518221604, ..."
3,RANIS KDS,[KRUPUK BAWANG RANIS KDS ],Malang,['KDS Cantonese Restaurant'],"[[-0.004616941175124867, -0.033262963255641514..."
4,Winarti Snack,[Kacang Goreng Winarti Snack],Surabaya,['Nathania Snack House'],"[[-0.023377033971692975, -0.013525635448501444..."
...,...,...,...,...,...
2827,virinda food,"[Rolade, Sempol]",Malang,['Javanine Resto'],"[[-0.04503288453259202, -0.04508640205255267, ..."
2828,wouwcake,"[risoles wouwcake, kue tar wouwcake, kroket ke...",Surabaya,['Warung wong lue (WWL)'],"[[-0.04631182359458489, -0.01782587469255922, ..."
2829,yuk tri,"[kue nastar, kue kastengel, kue putri salju]",Malang,['Trimurti Resto'],"[[0.006737446523112387, 0.007663352942509597, ..."
2830,zara,[stik keju zara],Malang,['Lafayette Coffee & Eatery'],"[[-0.05076225249692837, -0.05180014440515231, ..."


In [ ]:
# 2. Neo4j connection & similarity helpers
# -----------------------------------------------------------------------------
drv = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

def find_similar_restaurants_by_embedding(alts_embs: list[list[float]], threshold=0.6, topk=2):
    ids = []
    with drv.session() as ses:
        for emb in alts_embs:
            # Kueri ini mengembalikan properti 'r_id' dari node
            rows = ses.run("""
                CALL db.index.vector.queryNodes('restaurant_emb_small', $topk, $emb)
                YIELD node, score
                WHERE score >= $threshold
                OPTIONAL MATCH (other:Restaurant)-[s:SIMILAR_TO_SMALL]->(node)
                    WHERE s.score >= $threshold
                RETURN DISTINCT node.r_id AS id, collect(other.r_id) AS linked_ids
                """, emb=emb, threshold=threshold, topk=topk)
            for r in rows:
                if r["id"] is not None:
                    ids.append(r["id"])
                ids.extend(r["linked_ids"])
    return list(set(ids))

# Fungsi jaccard menerima list berisi string
def jaccard(list1: list[str], list2: list[str]) -> float:
    """Menghitung Jaccard similarity antara dua list string."""
    set1 = {item.lower().strip() for item in list1}
    set2 = {item.lower().strip() for item in list2}
    
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    return intersection / union if union != 0 else 0.0

In [ ]:
# 3. Overclaim checker
# -----------------------------------------------------------------------------
def check_overclaim(menu_names: list[str], alts_embs: list[list[float]]):
    # 1) look up their node IDs in Neo4j
    rids_count = 1
    for i in range(10):
        rids = find_similar_restaurants_by_embedding(alts_embs)
        print(f"Number of attempts to find similar restaurants by emb: {rids_count}")
        if rids: break
        rids_count += 1
        sleep(randint(1, 2))

    # 2) compare each menu_item list against your query list
    issues = []
    max_jaccard_score = 0.0
    # Variabel untuk menyimpan data perbandingan terbaik
    best_comp_name = None
    best_comp_menu = []
    
    with drv.session() as ses:
        for rid in rids:
            result = ses.run(
                """
                MATCH (r:Restaurant {r_id:$rid})
                OPTIONAL MATCH (r)-[:HAS_PRODUCT]->(m:Menu)
                RETURN r.restaurant AS restaurant_name, collect(m.menu) AS actual_menus
                """, rid=rid
            )
            rec = result.single()
            if rec and rec["restaurant_name"]:
                db_resto_name = rec["restaurant_name"]
                # Gunakan variabel ini untuk mengisi comp_menu nantinya
                actual_menus = rec["actual_menus"]

                split_menus = [item.split()[0] for item in actual_menus if item and item.strip()]

                score = jaccard(split_menus, menu_names)

                # Cek jika ini adalah skor tertinggi yang ditemukan sejauh ini
                if score > max_jaccard_score:
                    max_jaccard_score = score
                    best_comp_name = db_resto_name
                    # Simpan menu asli (bukan yang di-split)
                    best_comp_menu = actual_menus

                if score < 0.3:
                    issues.append({
                        "restaurant_id": rid,
                        "restaurant_name": db_resto_name,
                        "jaccard_score": score
                    })

    # Mengembalikan issues, score tertinggi, nama, dan menu perbandingan terbaik
    return issues, max_jaccard_score, best_comp_name, best_comp_menu

# Fungsi evaluate_row mengembalikan tuple (status, score, comp_name, comp_menu)
def evaluate_row(menus: list[str], emb: list[list[float]]) -> tuple[str, float, str | None, list[str]]:
    """
    Mengevaluasi satu baris data, menentukan status berdasarkan Jaccard score akhir.
    """
    # Tangkap semua nilai yang dikembalikan oleh check_overclaim
    issues, score, comp_name, comp_menu = check_overclaim(menus, emb)
    
    status = "overclaim" if score < 0.3 else "aman"
    
    # Kembalikan semua data yang relevan
    return status, score, comp_name, comp_menu

In [ ]:
# 4. Run on the "df" DataFrame
# -----------------------------------------------------------------------------
hasil = []
# Looping menangani score dan data perbandingan
for idx, row in df.iterrows():
    query_menus = [i.split()[0] for i in row["menu"] if i and i.strip()]
    
    # Tangkap semua nilai yang dikembalikan oleh evaluate_row
    status, score, comp_restaurant, comp_menu = evaluate_row(query_menus, row["embedding"])
    
    # Kondisi untuk append
    # Hanya tambahkan ke 'hasil' jika 'comp_restaurant' DAN 'comp_menu' tidak kosong.
    if comp_restaurant and comp_menu:
        hasil.append({
            "index": idx,
            "restaurant": row["restaurant"],
            "menu": row["menu"],
            "city": row["city"],
            "comp_restaurant": comp_restaurant,
            "comp_menu": comp_menu,
            "score": score,
            "status": status
        })
        print(f"Data valid ditemukan dan ditambahkan pada baris {idx+1} ✅ Total hasil sekarang: {len(hasil)}.")
    else:
        print(f"Data perbandingan kosong, baris {idx+1} dilewati ❌ Total hasil sekarang: {len(hasil)}.")

df_hasil = pd.DataFrame(hasil)
df_hasil

Number of attempts to find similar restaurants by emb: 1
Data perbandingan kosong, baris 1 dilewati ❌ Total hasil sekarang: 0.
Number of attempts to find similar restaurants by emb: 1
Data perbandingan kosong, baris 2 dilewati ❌ Total hasil sekarang: 0.
Number of attempts to find similar restaurants by emb: 1
Data perbandingan kosong, baris 3 dilewati ❌ Total hasil sekarang: 0.
Number of attempts to find similar restaurants by emb: 1
Data perbandingan kosong, baris 4 dilewati ❌ Total hasil sekarang: 0.
Number of attempts to find similar restaurants by emb: 1
Data valid ditemukan dan ditambahkan pada baris 5 ✅ Total hasil sekarang: 1.
Number of attempts to find similar restaurants by emb: 1
Data perbandingan kosong, baris 6 dilewati ❌ Total hasil sekarang: 1.
Number of attempts to find similar restaurants by emb: 1
Data perbandingan kosong, baris 7 dilewati ❌ Total hasil sekarang: 1.
Number of attempts to find similar restaurants by emb: 1
Data valid ditemukan dan ditambahkan pada baris

,index,restaurant,menu,city,comp_restaurant,comp_menu,score,status
0,4,Winarti Snack,[Kacang Goreng Winarti Snack],Surabaya,Cemilan Snack Medan,"[Sumpia 450 Gram, Kerupuk Kentang Jazz, Kacang...",0.083333,overclaim
1,7,-,"[Buntut Sapi, Daging Sapi, Jerohan Sapi, Kepal...",KAB. PROBOLINGGO,Sate Kuda Sumbawa,"[Susu Kuda Sumbawa 1 Botol 600ml, Nasi Putih, ...",0.055556,overclaim
2,14,77 DUA PUTRA,[KERUPUK IKAN TENGIRI ],Malang,"Warung Kongdhe, Bubutan","[Nasi Sayur Asem, Kerupuk Puli, Nasi Penyetan,...",0.062500,overclaim
3,15,99,"[Kulit Pangsit 99, MIE BASAH 99, Kulit pangsit...",Malang,Warung Teguh,"[Bakso Bakar Dan Tahu Bakar, Tela Tela, Tahu W...",0.111111,overclaim
4,33,AGEN SOSIS ALBY,"[BUMBU BAKSO, BUMBU TOMYAM, Sambal Bumbu, Samb...",KAB. KUTAI KARTANEGARA,"Markas Ranjau, Mojosari","[Klappertaart Durian, Mie Aceh Original Frozen...",0.016949,overclaim
...,...,...,...,...,...,...,...,...
1082,2817,spesialis lumpia alvaro,"[Lumpia Rebung Ori, Lumpia Rebung Ayam, Lumpia...",Malang,"Bakso Biru, Sidoarjo","[Bakso Jamur, Jus Alpukat, Lontong, Jeruk Ange...",0.062500,overclaim
1083,2818,sri rejeki,"[ote- ote sri rejeki, pisang goreng, tempe isi...",Malang,Warung Teguh,"[Bakso Bakar Dan Tahu Bakar, Tela Tela, Tahu W...",0.100000,overclaim
1084,2819,sweet cookies,"[kue nastar, thumprint choco, kastengel, Palm ...",Malang,"Raihan Bakery, Tembung","[Spong Kacang Hijau, Marmer Cake Bulat, Kacang...",0.039216,overclaim
1085,2823,telur gulung mak ti,"[telur gulung, nasi goreng, pisang keju]",Kabupaten Gresik,"Bakso Titoti, Parung","[Es Kelapa, Bakso Telur, Ayam Kampung Goreng K...",0.040000,overclaim


In [49]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [7]:
df_hasil.to_excel("df_hasil_1kata_1087_rows_1536dim.xlsx")